In [46]:
import Turing_Pipelines_I  # Importa todo el código del notebook convertido en script
import os
import joblib

# Acceder a las variables del módulo importado
X_train = Turing_Pipelines_I.X_train  # Obtener los datos originales de entrenamiento
y_train = Turing_Pipelines_I.y_train  # Obtener las etiquetas de entrenamiento
preprocessor = Turing_Pipelines_I.preprocessor  # Obtener el preprocesador
model_path = Turing_Pipelines_I.model_path  # Obtener la ruta del modelo guardado

# Verificar si el modelo existe antes de cargarlo
if os.path.exists(model_path):
    loaded_model = joblib.load(model_path)
    print("Modelo cargado correctamente.")
else:
    raise FileNotFoundError(f"Error: El modelo no existe en {model_path}")

# Preprocesar los datos si no están preprocesados
if "X_train_preprocessed" in dir(Turing_Pipelines_I):
    X_train_preprocessed = Turing_Pipelines_I.X_train_preprocessed
else:
    X_train_preprocessed = preprocessor.transform(X_train)  # Aplicar el preprocesador

# Verificar si el modelo cargado es un Pipeline y acceder al clasificador si es necesario
if hasattr(loaded_model, "named_steps") and "classifier" in loaded_model.named_steps:
    real_model = loaded_model.named_steps["classifier"]
else:
    real_model = loaded_model  # Si no es un Pipeline, usar el modelo directamente

# Verificar si el modelo está entrenado
if not hasattr(real_model, "classes_"):  # Verifica si el modelo tiene clases entrenadas
    print("El modelo no está entrenado. Reentrenando...")
    pipeline = Turing_Pipelines_I.pipeline  # Asegúrate de que el pipeline esté disponible
    pipeline.fit(X_train, y_train)
    joblib.dump(pipeline, model_path)  # Guarda el modelo nuevamente
    real_model = pipeline.named_steps["classifier"]

X_train_preprocessed = preprocessor.transform(X_train)  # Aplicar el preprocesador

# Hacer una predicción con los datos de entrenamiento
try:
    sample_prediction = real_model.predict(X_train_preprocessed[:5])
    print(f"Predicción de prueba del modelo cargado: {sample_prediction}")
except Exception as e:
    print(f"Error al hacer la predicción: {e}")

Modelo cargado correctamente.
Predicción de prueba del modelo cargado: [0 0 0 1 0]
